In [124]:
# Run in terminal or command prompt
# python3 -m spacy download en

import numpy as np
import pandas as pd
import re, nltk, spacy, gensim

# Sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint

In [125]:
df = pd.concat(pd.read_excel("C:\\Users\\v-savrav\\Documents\\SampleDataset2.xlsx", sheet_name=None), ignore_index=True)

In [126]:
df.columns

Index(['PrimaryTopic', 'secondarytopic', 'description'], dtype='object')

In [127]:
# Convert to list
data = df["description"].values.tolist()

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['hi', 'team', 'please', 'cancel', 'the', 'first', 'submission', 'of', 'er', 'as', 'the', 'incorrect', 'receipts', 'were', 'attached', 'however', 'am', 'now', 'going', 'to', 'submit', 'the', 'correct', 'receipts', 'to', 'the', 'er', 'thank', 'you', 'regards', 'renee', 'bradshaw']]


In [128]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
#stop_words.extend(['from', 'subject', 're', 'check','apars_apar','error'])
stop_words.extend(['close', 'hour','subject','desk','find', 'name','thank','thanks','send','com','prohelp','regards','regard','email','information','message','team',
                   'attachment','mail','check','apars_apar','error','attached','www','hi','cc','sent','date','use','try','query'])

In [129]:
def remove_stopwords(texts):
    return [[word for word in gensim.utils.simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

data_words_nostops = remove_stopwords(data_words)

In [130]:
data_words_nostops[2]

['please',
 'correction',
 'docs',
 'reference',
 'old',
 'doc',
 'reversal',
 'doc',
 'correct',
 'docs',
 'dinesh',
 'kumar',
 'accenture',
 'skype',
 'luxemburg',
 'ap',
 'friday',
 'june',
 'pm',
 'deepalakshmi',
 'murugesan',
 'accenture',
 'demuru',
 'microsoft',
 'hariram',
 'krishnamoorthy',
 'accenture',
 'hakris',
 'microsoft',
 'hariram',
 'krishnamoorthy',
 'accenture',
 'hakris',
 'microsoft',
 'qing',
 'fang',
 'accenture',
 'qifang',
 'microsoft',
 'anna',
 'talarek',
 'accenture',
 'antal',
 'microsoft',
 'dinesh',
 'kumar',
 'accenture',
 'diku',
 'microsoft',
 'dinesh',
 'kumar',
 'accenture',
 'diku',
 'microsoft',
 'ranjitha',
 'ramalingam',
 'accenture',
 'ranram',
 'microsoft',
 'ranjitha',
 'ramalingam',
 'accenture',
 'ranram',
 'microsoft',
 'record',
 'skype',
 'recskype',
 'microsoft',
 'vijay',
 'kumar',
 'accenture',
 'vikuma',
 'microsoft',
 'vijay',
 'kumar',
 'accenture',
 'vikuma',
 'microsoft',
 'tkt',
 'topps',
 'digital',
 'services',
 'invoices',
 '

In [143]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# Run in terminal: python3 -m spacy download en
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])
#nlp = spacy.load("en_core_web_sm")
# Do lemmatization keeping only Noun, Adj, Verb, Adverb
data_lemmatized = lemmatization(data_words_nostops, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:2])

['cancel first submission incorrect receipt however go submit correct receipt renee bradshaw', 'regard number follow regard number correta customer regard number regard number regard number kindly provide valid po number invoice']


In [144]:
vectorizer = CountVectorizer(analyzer='word',       
                             min_df=10,                        # minimum reqd occurences of a word 
                             stop_words='english',             # remove stop words
                             lowercase=True,                   # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
                             # max_features=50000,             # max number of uniq words
                            )

data_vectorized = vectorizer.fit_transform(data_lemmatized)

In [145]:
# Build LDA Model
lda_model = LatentDirichletAllocation(n_components=5,               # Number of topics
                                      max_iter=10,               # Max learning iterations
                                      learning_method='online',   
                                      random_state=100,          # Random state
                                      batch_size=128            # n docs in each learning iter
                                     )
lda_output = lda_model.fit_transform(data_vectorized)

print(lda_model)  # Model attributes

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='online', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=100, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)


In [146]:
# Define Search Param
search_params = {'n_components': [4,5,6], 'learning_decay': [.5, .7, .9]}

# Init the Model
lda = LatentDirichletAllocation()

# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)

# Do the Grid Search
model.fit(data_vectorized)

GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=LatentDirichletAllocation(batch_size=128,
                                                 doc_topic_prior=None,
                                                 evaluate_every=-1,
                                                 learning_decay=0.7,
                                                 learning_method='batch',
                                                 learning_offset=10.0,
                                                 max_doc_update_iter=100,
                                                 max_iter=10,
                                                 mean_change_tol=0.001,
                                                 n_components=10, n_jobs=None,
                                                 perp_tol=0.1,
                                                 random_state=None,
                                                 topic_word_prior=None,
                                   

In [154]:
# Best Model
best_lda_model = model.best_estimator_

# Model Parameters
print("Best Model's Params: ", model.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(data_vectorized))

Best Model's Params:  {'learning_decay': 0.7, 'n_components': 4}
Best Log Likelihood Score:  -57335.59550835433
Model Perplexity:  243.97583133663548


In [148]:
# Create Document - Topic Matrix
lda_output = best_lda_model.transform(data_vectorized)

# column names
topicnames = ["Topic" + str(i) for i in range(best_lda_model.n_components)]

# index names
docnames = ["Doc" + str(i) for i in range(len(data))]

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic

# Styling
def color_green(val):
    color = 'green' if val > .1 else 'black'
    return 'color: {col}'.format(col=color)

def make_bold(val):
    weight = 700 if val > .1 else 400
    return 'font-weight: {weight}'.format(weight=weight)

# Apply Style
df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
df_document_topics


,Topic0,Topic1,Topic2,Topic3,dominant_topic
Doc0,0.89,0.04,0.04,0.04,0
Doc1,0.02,0.95,0.02,0.02,1
Doc2,0.01,0.41,0.57,0.01,2
Doc3,0,0.56,0.44,0,1
Doc4,0.03,0.03,0.72,0.23,2
Doc5,0.03,0.37,0.58,0.03,2
Doc6,0.01,0.96,0.01,0.01,1
Doc7,0.36,0.2,0.43,0.01,2
Doc8,0.62,0,0.31,0.06,0
Doc9,0.25,0.25,0.25,0.25,0


In [149]:
df_topic_distribution = df_document_topic['dominant_topic'].value_counts().reset_index(name="Num Documents")
df_topic_distribution.columns = ['Topic Num', 'Num Documents']
df_topic_distribution

,Topic Num,Num Documents
0,2,351
1,0,312
2,3,182
3,1,155


In [150]:
# Topic-Keyword Matrix
df_topic_keywords = pd.DataFrame(best_lda_model.components_)

# Assign Column and Index
df_topic_keywords.columns = vectorizer.get_feature_names()
df_topic_keywords.index = topicnames

# View
df_topic_keywords.head()

,able,absolutely,accenture,accept,acceptable,access,account,accountant,accounting,action,...,way,web,website,week,wire,work,worldwide,write,wrong,year
Topic0,7.345416,12.233269,51.362664,0.261953,13.682802,7.100803,269.367992,3.400850,0.254806,94.256781,...,0.270661,5.561716,0.250037,9.255352,16.319176,5.474031,0.254716,1.263846,4.538342,0.258123
Topic1,6.992187,0.250510,19.516617,22.788755,0.256936,21.857833,79.929167,0.260804,0.252822,29.544489,...,4.839267,0.251439,4.735198,4.027438,18.162772,1.188398,0.441890,15.167271,0.254363,14.295571
Topic2,38.258293,0.257943,19.862814,5.199158,47.812104,40.202315,315.784364,0.262412,0.257313,63.853160,...,39.404189,0.277530,10.905973,11.855956,0.254151,21.871658,0.264059,22.306669,6.949571,22.727687
Topic3,8.404105,0.258278,0.257905,1.750134,5.248158,36.839049,90.918477,8.075934,22.235058,0.345570,...,7.485883,4.909314,4.108793,6.861255,0.263901,43.465913,13.039335,0.262213,0.257724,6.718619


In [151]:
# Show top n keywords for each topic
def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

topic_keywords = show_topics(vectorizer=vectorizer, lda_model=best_lda_model, n_words=15)        

# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14
Topic 0,expense,report,payment,account,receipt,receive,payable,submit,help,reimbursement,require,finance,need,outsource,assistance
Topic 1,payment,number,receive,intend,contain,recipient,account,notify,immediately,confidential,request,bank,attachment,contact,copy
Topic 2,invoice,account,request,payable,payment,need,response,support,hour,help,receive,know,status,let,ticket
Topic 3,pay,service,account,manager,good,card,corporate,office,credit,follow,support,know,receive,kindly,refund


In [139]:
# Show top n keywords for each topic
def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

topic_keywords = show_topics(vectorizer=vectorizer, lda_model=best_lda_model, n_words=15)        

# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14
Topic 0,receive,payment,intend,privacy,tax,number,partner,contain,recipient,request,service,company,immediately,notify,address
Topic 1,expense,report,submit,account,receipt,payment,require,request,help,process,need,action,employee,receive,reimbursement
Topic 2,payment,invoice,account,payable,response,hour,finance,status,receive,need,support,expense,escalate,update,number


In [140]:
# try to append back to original df
final_list = []
for i in range(len(df_document_topic)):
    final_list.append(df_document_topic["dominant_topic"][i])
#final_list

In [141]:
df ["dominant_topic"] = final_list

In [142]:
df

,PrimaryTopic,secondarytopic,description,dominant_topic
0,NaN,NaN,"Hi Team, Please cancel the first submission o...",1
1,NaN,NaN,"Hi AP,Please discard the email below and use t...",2
2,NaN,NaN,"Hi Team, Please find correction docs for your ...",2
3,NaN,NaN,Hello Prohelp and US AP Escalations <usapesc@m...,0
4,NaN,NaN,"Hi team, I’m attaching 3 invoices from Wunderm...",2
5,NaN,NaN,Please find attached the Invoice and POE for P...,0
6,NaN,NaN,Hi Prohelp- I’m wondering if you could direct...,0
7,NaN,NaN,It does thankyou&#128522; Many thanks Helen ...,2
8,NaN,NaN,Thank you Kiran! Sorry but I do not understan...,2
9,NaN,NaN,Hea klient Kirjaga on kaasas PDF-formaadis arv...,0


In [28]:
df.to_excel("topicmodeling.xlsx")

In [1]:
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('model5.gensim')
import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

NameError: name 'gensim' is not defined